In [284]:
import pandas as pd
import os

In [285]:
variant_folder = '/home/laura/ANALYSIS/VARIANT_CALLING/ABSCESSUS_ATCC/Variants/'

query_sample = 'ALM13739611-1042'
query = os.path.join(variant_folder + '/' + query_sample, 'snps.all.ivar.tsv')

output_directory = '/home/laura/IMPORTANTE/Papers/Abscessus/Intrapaciente/1042/Heterocigotos/End/Genotipo13/'
output_file = os.path.join(output_directory, f"{query_sample}_HTZ_comparison.xlsx")
output_hq_file = os.path.join(output_directory, f"{query_sample}_HQ_comparison.xlsx")

initial_samples_file = os.path.join(output_directory, 'Aislados')

coverage_directory = '/home/laura/ANALYSIS/VARIANT_CALLING/ABSCESSUS_ATCC/Stats/Coverage/'


In [286]:
query_df = pd.read_csv(query, sep='\t')

query_htz = query_df[(query_df.ALT_DP >= 20) & (query_df.ALT_FREQ < 0.7) & (
    query_df.ALT_FREQ >= 0.2) & (query_df.TYPE == 'snp')].reset_index()

query_htz = query_htz[['REGION', 'POS', 'REF',
                               'ALT', 'TOTAL_DP', 'REF_FREQ', 'ALT_FREQ', 'OLDVAR']]


query_hq = query_df[(query_df.TOTAL_DP >= 20) & (query_df.ALT_FREQ >= 0.8) & (query_df.TYPE == 'snp')].reset_index()

query_hq = query_hq[['REGION', 'POS', 'REF',
                               'ALT', 'TOTAL_DP', 'REF_FREQ', 'ALT_FREQ', 'OLDVAR']]

In [287]:
with open(initial_samples_file, 'r') as f:
    initial_samples = [line.strip() for line in f]

In [288]:
# Inicializar el DataFrame combinado
combined_htz_comparison = query_htz.copy()

combined_hq_comparison = query_hq.copy()

In [289]:
def fill_missing_values(comparison, missing_rows, coverage_file, initial_sample):
    # Leer el archivo de cobertura como un DataFrame
    coverage_df = pd.read_csv(coverage_file, sep='\t', header=None, names=['Genoma', 'POS', 'DP'])
    
    # Crear un diccionario para almacenar los valores de profundidad por posición
    coverage_dict = dict(zip(coverage_df['POS'], coverage_df['DP']))
    
    # Iterar sobre las filas faltantes y extraer los valores correspondientes del 'coverage_dict'
    for index, row in missing_rows.iterrows():
        pos = row['POS']
        if pd.notnull(pos) and pos in coverage_dict:
            coverage_value = coverage_dict[pos]
            comparison.at[index, ('DP_' + initial_sample)] = coverage_value
    
    # Exportar el DataFrame a Excel
    return comparison

In [290]:
# Iterar sobre las muestras iniciales
for initial_sample in initial_samples:
    initial = os.path.join(variant_folder + '/' + initial_sample, 'snps.all.ivar.tsv')
    coverage_file = os.path.join(coverage_directory, initial_sample + '.cov')

    initial_df = pd.read_csv(initial, sep='\t')
    initial_df = initial_df[['POS', 'TOTAL_DP', 'ALT_DP', 'ALT_FREQ']] # 'ALT', 
    initial_df = initial_df.rename(columns={'TOTAL_DP': f'DP_{initial_sample}', 'ALT_DP': f'ALT_DP_{initial_sample}', 'ALT_FREQ': f'ALT_FREQ_{initial_sample}'}) # 'ALT': f'ALT_{initial_sample}',

    # Realizar un merge en lugar de una concatenación
    combined_htz_comparison = combined_htz_comparison.merge(initial_df, how='left', on='POS')


    # Buscar los valores faltantes en la columna 'DP_' + initial_sample
    missing_values = combined_htz_comparison[f'DP_{initial_sample}'].isna()

        # Filtrar las filas con valores faltantes
    missing_rows = combined_htz_comparison[missing_values]

    fill_missing_values(combined_htz_comparison, missing_rows, coverage_file, initial_sample)

# Exportar el DataFrame combinado a Excel
combined_htz_comparison.to_excel(output_file, index=False)

In [291]:
# Iterar sobre las muestras iniciales
for initial_sample in initial_samples:
    initial = os.path.join(variant_folder + '/' + initial_sample, 'snps.all.ivar.tsv')
    coverage_file = os.path.join(coverage_directory, initial_sample + '.cov')

    initial_df = pd.read_csv(initial, sep='\t')
    initial_df = initial_df[['POS', 'ALT', 'TOTAL_DP', 'ALT_DP', 'ALT_FREQ']]
    initial_df = initial_df.rename(columns={'ALT': f'ALT_{initial_sample}',
                                            'TOTAL_DP': f'DP_{initial_sample}', 'ALT_DP': f'ALT_DP_{initial_sample}', 'ALT_FREQ': f'ALT_FREQ_{initial_sample}'})

    # Realizar un merge en lugar de una concatenación
    combined_hq_comparison = combined_hq_comparison.merge(initial_df, how='left', on='POS')


    # Buscar los valores faltantes en la columna 'DP_' + initial_sample
    missing_values = combined_hq_comparison[f'DP_{initial_sample}'].isna()

        # Filtrar las filas con valores faltantes
    missing_rows = combined_hq_comparison[missing_values]

    fill_missing_values(combined_hq_comparison, missing_rows, coverage_file, initial_sample)

# Exportar el DataFrame combinado a Excel
combined_hq_comparison.to_excel(output_hq_file, index=False)